In [20]:
import numpy as np 
import pandas as pd 
import os
from multiprocessing import Pool

import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [2]:
def g_score(true_positives, false_positives, false_negatives):
    return max(0, true_positives - false_positives) / (true_positives + false_negatives + 1e-7)  # Added epsilon to avoid division by zero

def rank_score(g_scores):
    return sum(1**g for g in g_scores)


In [3]:
class GCNLayer:
    def __init__(self, input_dim, output_dim):
        self.weights = initialize_weights(input_dim, output_dim)
        self.grads = np.zeros_like(self.weights)

    def relu(self, x):
        return np.maximum(0, x)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def forward(self, A, X):
        return self.relu(A @ X @ self.weights)
    
    def backward(self, d_loss):
        # Compute gradients using the chain rule
        self.grads = d_loss  # Update this as per your specific loss and activation
        return self.grads

class GCN:
    def __init__(self, input_dim, hidden_dim, output_dim):
        self.gcn1 = GCNLayer(input_dim, hidden_dim)
        self.gcn2 = GCNLayer(hidden_dim, output_dim)

    def forward(self, A, X):
        self.H1 = self.gcn1.forward(A, X)
        self.H2 = self.gcn2.forward(A, self.H1)
        return self.H2

    def backward(self, d_loss):
        # Backward pass for the second layer
        d_loss2 = self.gcn2.backward(d_loss)
        
        # Backward pass for the first layer
        d_loss1 = self.gcn1.backward(d_loss2)
        
    def update_weights(self, learning_rate):
        self.gcn1.weights -= learning_rate * self.gcn1.grads
        self.gcn2.weights -= learning_rate * self.gcn2.grads


In [30]:
def create_datasets_core_logic(args):
    idx, alarm, causal = args
    
    # Initialize DataFrame with lists of zeros
    init_data = [[np.zeros(2**8, dtype=np.uint8).tolist()] * 2**22 for _ in range(2**6)]
    df = pd.DataFrame(init_data, index=np.arange(2**6, dtype=np.uint8), columns=np.arange(2**22, dtype=np.uint32))

    for alarm_id in df.index:
        rows = alarm[alarm['alarm_id'] == alarm_id]
        for _, row in rows.iterrows():
            for t in range(row['start_timestamp'], row['end_timestamp']):
                df.at[alarm_id, t][row['device_id']] = 1

    # Add a column for causal matrix rows
    df['causal'] = np.zeros(2**6, dtype = np.uint8)
    df['causal'] = [causal[i] for i in causal]

    # Save DataFrame
    df.to_pickle(f"dataset_{idx}.pkl")
    
def create_datasets(dataset_list):
    pool = True
    if pool:
        with Pool() as pool:
            pool.map(create_datasets_core_logic, [(idx, alarm, causal) for idx, (alarm, causal) in enumerate(dataset_list)])
    else:
        for idx, (alarm, causal) in enumerate(dataset_list):
            create_datasets_core_logic((idx, alarm, causal))

In [17]:
alarm1 = pd.read_csv('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_1/alarm.csv')
causal1 = np.load('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_1/causal_prior.npy' , allow_pickle = True)

alarm2 = pd.read_csv('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_2/alarm.csv')
causal2 = np.load('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_2/causal_prior.npy' , allow_pickle = True)

alarm3 = pd.read_csv('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_3/alarm.csv')
causal3 = np.load('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_3/causal_prior.npy' , allow_pickle = True)

alarm4 = pd.read_csv('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_4/alarm.csv')
causal4 = np.load('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_4/causal_prior.npy' , allow_pickle = True)

rca1 = pd.read_csv('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_1/rca_prior.csv')
topology1 = np.load('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_1/topology.npy' , allow_pickle = True)
rca2 = pd.read_csv('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_2/rca_prior.csv')
topology2 = np.load('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_2/topology.npy' , allow_pickle = True)
rca3 = pd.read_csv('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_3/rca_prior.csv')
topology3 = np.load('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_3/topology.npy' , allow_pickle = True)

dataset_list = [(alarm1, causal1), (alarm2, causal2), (alarm3, causal3), (alarm4, causal4)]

In [ ]:
create_datasets(dataset_list)

In [6]:

times = {}
alarms = {}
devices = {}
times_list = []
alarms_list = []
devices_list = []
for i, (alarm, _) in enumerate(dataset_list):
    times[i] = list(alarm['start_timestamp'].unique())+list(alarm['end_timestamp'].unique())
    times_list += times[i]
    alarms[i] = list(alarm['alarm_id'].unique())
    alarms_list += alarms[i]
    devices[i] = list(alarm['device_id'].unique())
    devices_list += devices[i]
times = len(set(times_list))
alarms = len(set(alarms_list))
devices = len(set(devices_list))
print(times)
print(alarms)
print(devices)
print()
print(len(alarm1['alarm_id'].unique()))
print(len(alarm2['alarm_id'].unique()))
print(len(alarm3['alarm_id'].unique()))
print(len(alarm4['alarm_id'].unique()))
print()
print(len(alarm1['device_id'].unique()))
print(len(alarm2['device_id'].unique()))
print(len(alarm3['device_id'].unique()))
print(len(alarm4['device_id'].unique()))
print()
print(len(set(list(alarm1['start_timestamp'].unique())+list(alarm1['end_timestamp'].unique()))))
print(len(set(list(alarm2['start_timestamp'].unique())+list(alarm2['end_timestamp'].unique()))))
print(len(set(list(alarm3['start_timestamp'].unique())+list(alarm3['end_timestamp'].unique()))))
print(len(set(list(alarm4['start_timestamp'].unique())+list(alarm4['end_timestamp'].unique()))))
# Generate the dataset
###new_dataset = create_dataset(dataset_list)

616313
49
100

39
49
31
30

35
42
39
100

203619
147963
241340
223427


In [15]:
print(sorted(list(alarm1['alarm_id'].unique())))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]


In [8]:
print(max(list(alarm4['end_timestamp'])))

2392489


In [9]:
print(len(list(alarm4['start_timestamp'].unique())))

117367


In [13]:
alarm4


,alarm_id,device_id,start_timestamp,end_timestamp
0,27,68,3,18067
1,16,58,3,18071
2,16,2,10,18082
3,27,58,16,18086
4,27,2,24,18088
...,...,...,...,...
126798,21,24,2332760,2333009
126799,27,17,2332762,2350823
126800,22,52,2332770,2332897
126801,20,33,2332789,2332821
